# Agents

In [1]:
# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Literal, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [6]:
go_up_from_current_directory(go_up=2)

from QA_and_RAG import PACKAGE_ROOT_PATH
from config import settings, config


PACKAGE_ROOT_PATH

/Users/neidu/Desktop/Projects/Personal/My_Projects/Gen-AI-Projects


PosixPath('/Users/neidu/Desktop/Projects/Personal/My_Projects/Gen-AI-Projects/QA_and_RAG')

In [5]:
import os
import langchain
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from langchain_community.agent_toolkits import create_sql_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    RunnableLambda,
    RunnablePassthrough,
    RunnableSequence,
)
from langchain_openai import ChatOpenAI


langchain.debug = True

In [7]:
from omegaconf import OmegaConf, DictConfig


# config: DictConfig = OmegaConf.load("../../config/config.yaml")

print(OmegaConf.to_yaml(config, resolve=True))

QA_and_RAG:
  path: /Users/neidu/Desktop/Projects/Personal/My_Projects/Gen-AI-Projects//QA_and_RAG
  chinook_db_path: /Users/neidu/Desktop/Projects/Personal/My_Projects/Gen-AI-Projects//QA_and_RAG/data/sql/chinook.db
  stored_sql_db_path: /Users/neidu/Desktop/Projects/Personal/My_Projects/Gen-AI-Projects//QA_and_RAG/data/flat_files/stored_data.db
  uploaded_files_path: /Users/neidu/Desktop/Projects/Personal/My_Projects/Gen-AI-Projects//QA_and_RAG/data/for_upload
  uploaded_db_path: /Users/neidu/Desktop/Projects/Personal/My_Projects/Gen-AI-Projects//QA_and_RAG/data/for_upload/db_path/uploaded_data.db
  llm:
    model: gpt-4o-mini
    temperature: 0.0
  sql_agent_prompt: 'Given an input question, create a syntactically correct {dialect}
    query to run to help find the answer. Unless the user specifies in his question
    a specific number of examples they wish to obtain, always limit your query to
    at most {top_k} results. You can order the results by a relevant column to return
   

In [8]:
file_path: str = str(PACKAGE_ROOT_PATH / "/data/sql/chinook.db")
file_path

'/data/sql/chinook.db'

In [9]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=settings.OPENAI_API_KEY, temperature=0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1249eae70>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x124a18b00>, root_client=<openai.OpenAI object at 0x12424f860>, root_async_client=<openai.AsyncOpenAI object at 0x1249eaed0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

### Old Approach (Previous Versions of LangChain)

In [10]:
prompt_1: str = """Based on the database query result, provide a complete sentence
    answer to the user''s question.

    Question: {question}
    SQL Result: {result}
    Answer:
    """

In [11]:
def _qa_sql_agent(db_path: str, prompt: str, message: str, llm: Any) -> str:
    """Execute SQL queries and format answers using a language model.

    Parameters
    ----------
    db_path : str
        Path to the SQLite database file.
    prompt : str
        Template string for formatting the answer.
    message : str
        User question to be answered.
    llm : Any
        Language model instance for generating SQL queries and answers.

    Returns
    -------
    str
        Formatted response to the user's question.
    """
    db: SQLDatabase = SQLDatabase.from_uri(
        f"sqlite:///../{db_path}"
    )  # for jupyter notebook
    print(f"DB Dialect: {db.dialect}\nTable Names: {db.get_usable_table_names()}")

    exec_query: QuerySQLDatabaseTool = QuerySQLDatabaseTool(db=db)
    write_query: RunnableSequence = create_sql_query_chain(llm, db=db)

    answer_prompt: PromptTemplate = PromptTemplate.from_template(template=prompt)

    answer: RunnableSequence = answer_prompt | llm | StrOutputParser()

    def execute_and_format(data: dict[str, str]) -> str:
        """Execute SQL query and format the answer.

        Parameters
        ----------
        data : dict[str, str]
            Dictionary containing 'query' and 'question' keys with string values.
            The 'query' value may optionally start with 'SQLQuery:'.

        Returns
        -------
        str
            Formatted answer based on the question and query result.
        """
        # Execute query
        sql: str = data["query"]
        print("===" * 20)
        print("sql: ", sql)
        if (
            sql.startswith("SQLQuery:")
            or sql.startswith("sql:")
            or sql.__contains__("")
        ):
            sql = (
                sql.replace("SQLQuery:", "")
                .replace("sql:", "")
                .replace("sql", "")
                .replace("```", "")
                .strip()
            )
        result: str = exec_query.invoke(sql)

        return answer.invoke({"question": data["question"], "result": result})

    chain: RunnableSequence = (
        RunnablePassthrough()
        # Add the key `query` to the input dict
        .assign(query=write_query)
        # Chain the output of the prev step with the next step
        .pipe(RunnableLambda(execute_and_format))
    )
    response: str = chain.invoke({"question": message})
    return response

In [12]:
fp: str = "../data/flat_files/titanic-data.csv"
df: pl.DataFrame = pl.read_csv(fp)

# print(df["BloodPressure"].mean())
df.filter(pl.col("sex").eq("male")).describe()
# df.head()

statistic,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
str,f64,f64,str,str,f64,f64,f64,str,f64,str,str,str,f64,str
"""count""",843.0,843.0,"""843""","""843""",658.0,843.0,843.0,"""843""",842.0,"""154""","""843""","""167""",113.0,"""457"""
"""null_count""",0.0,0.0,"""0""","""0""",185.0,0.0,0.0,"""0""",1.0,"""689""","""0""","""676""",730.0,"""386"""
"""mean""",2.372479,0.190985,null,null,30.585233,0.413998,0.247924,null,26.154601,null,null,null,160.39823,null
"""std""",0.811908,0.39331,null,null,14.280571,0.997928,0.708938,null,42.486877,null,null,null,95.035289,null
"""min""",1.0,0.0,"""Abbing, Mr. Anthony""","""male""",0.3333,0.0,0.0,"""110413""",0.0,"""A10""","""C""","""1""",1.0,"""?Havana, Cuba"""
"""25%""",2.0,0.0,null,null,21.0,0.0,0.0,null,7.875,null,null,null,79.0,null
"""50%""",3.0,0.0,null,null,28.0,0.0,0.0,null,12.275,null,null,null,155.0,null
"""75%""",3.0,0.0,null,null,39.0,1.0,0.0,null,26.55,null,null,null,255.0,null
"""max""",3.0,1.0,"""van Melkebeke, Mr. Philemon""","""male""",80.0,8.0,9.0,"""WE/P 5735""",512.3292,"""T""","""S""","""D""",322.0,"""Zurich, Switzerland"""


In [13]:
from enum import Enum

from QA_and_RAG.src.db_utils import SQLFromTabularData


class ChatType(Enum):
    qa_with_stored_sql_db = "q&a-with-stored-sql-db"
    qa_with_stored_flat_file_sql_db = "q&a-with-stored-flat-file-sql-db"
    qa_with_uploaded_flat_file_sql_db = "q&a-with-uploaded-flat-file-sql-db"


chat_type: Literal[
    ChatType.qa_with_stored_sql_db,
    ChatType.qa_with_stored_flat_file_sql_db,
    ChatType.qa_with_uploaded_flat_file_sql_db,
] = ChatType.qa_with_stored_sql_db.value
app_functionality: str = "chat"
message: str = "How many employees are there"
chatbot: list[str] = []
file_path: str = str(PACKAGE_ROOT_PATH / "/data/sql/chinook.db")
flat_file_path: str = str(PACKAGE_ROOT_PATH / "/data/flat_files/stored_data.db")

if app_functionality == "chat":
    if chat_type == ChatType.qa_with_stored_sql_db.value:
        # if os.path.exists(config.QA_and_RAG.chinook_db_path):
        if os.path.exists(f"../{file_path}"):  # for jupyter notebook
            response = _qa_sql_agent(
                db_path=file_path, prompt=prompt_1, message=message, llm=llm
            )
        else:
            chatbot.append(
                (
                    message,
                    f"SQL DB does not exist. Please first create the `chinook` db",
                )
            )
    # elif chat_type == "qa-with-flat-file-sql-db":
    elif chat_type == ChatType.qa_with_uploaded_flat_file_sql_db.value:
        _db_path: str = "../data/for_upload/uploaded_data.db"
        create_db: SQLFromTabularData = SQLFromTabularData(
            file_path="uploaded_data.csv",
            db_path=_db_path,
            table_name="your_table_name",
        )
        if os.path.exists(f"../{file_path}"):  # for jupyter notebook
            response = _qa_sql_agent(
                db_path=_db_path, prompt=prompt_1, message=message, llm=llm
            )
        else:
            chatbot.append(
                (
                    message,
                    f"SQL DB from does not exist. Please upload a valid `CSV`/`PARQUET` file.",
                )
            )

    elif chat_type == ChatType.qa_with_stored_flat_file_sql_db.value:
        if os.path.exists(f"../{file_path}"):  # for jupyter notebook
            response = _qa_sql_agent(
                db_path=file_path, prompt=prompt_1, message=message, llm=llm
            )
        else:
            chatbot.append(
                (
                    message,
                    f"SQL DB from does not exist. Please first create the `chinook` db",
                )
            )

    chatbot.append((message, response))

DB Dialect: sqlite
Table Names: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "How many employees are there"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<query>] Entering Chain run with input:
{
  "question": "How many employees are there"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<query> > chain:RunnableParallel<query>] Entering Chain run with input:
{
  "question": "How many employees are there"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<query> > chain:RunnableParallel<query> > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "How many employees are there"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<query> > chain:RunnableParallel<query> > chain:RunnableSequence > chain:RunnableAssign<input,table_info>] Entering Chain r

In [14]:
def get_database(db_path: str) -> SQLDatabase:
    db: SQLDatabase = SQLDatabase.from_uri(
        f"sqlite:///../{db_path}"
    )  # for jupyter notebook
    print(db.dialect)
    return db

In [15]:
db_path: str = "/data/flat_files/stored_data.db"
message: str = "What is the average blood pressure?"
message: str = "How many men survived in the Titanic?"

_qa_sql_agent(db_path=db_path, prompt=prompt_1, message=message, llm=llm)

DB Dialect: sqlite
Table Names: ['breast_cancer', 'diabetes', 'titanic']
[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "How many men survived in the Titanic?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<query>] Entering Chain run with input:
{
  "question": "How many men survived in the Titanic?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<query> > chain:RunnableParallel<query>] Entering Chain run with input:
{
  "question": "How many men survived in the Titanic?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<query> > chain:RunnableParallel<query> > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "How many men survived in the Titanic?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<query> > chain:RunnableParallel<query> > chain:RunnableSequence > chain:RunnableAssign<input,table_info>] Entering Chain run with input:
{
  "question": "How many men survived

'A total of 161 men survived the Titanic disaster.'

<br>

### Using Agent Executor (Old Approach)

In [16]:
db_path: str = "/data/flat_files/stored_data.db"
db: SQLDatabase = get_database(db_path=db_path)

agent_executor = create_sql_agent(
    llm=llm, db=db, agent_type="openai-tools", verbose=False
)
response = agent_executor.invoke(message)["output"]
response

sqlite
[chain/start] [chain:SQL Agent Executor] Entering Chain run with input:
{
  "input": "How many men survived in the Titanic?"
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:SQL Agent Executor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] 

'A total of 161 men survived the Titanic.'

### Modern Approach

In [17]:
from langchain import hub
from langchain_core.prompts.chat import ChatPromptTemplate


query_prompt_template: ChatPromptTemplate = hub.pull(
    "langchain-ai/sql-query-system-prompt"
)
# console.print(query_prompt_template.messages[0])
query_prompt_template.messages[0].pretty_print()

================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}


In [18]:
from typing import Annotated, TypedDict


chat_llm: ChatOpenAI = ChatOpenAI(
    model="gpt-4o-mini", api_key=settings.OPENAI_API_KEY, temperature=0
)


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = chat_llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}


def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = chat_llm.invoke(prompt)
    return {"answer": response.content}

In [19]:
state_data: State = State(
    # {"question": "How many men were there in the Titanic that survived?"}
    {"question": "What is the ratio of men to women that survived the Titanic?"}
)
query: str = write_query(state_data)["query"]
state_data["query"] = query

query_result: dict[str, Any] = execute_query(state_data)["result"]
state_data["result"] = query_result

generate_answer(state=state_data)

[chain/start] [prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "dialect": "sqlite",
  "top_k": 10,
  "table_info": "\nCREATE TABLE breast_cancer (\n\tid BIGINT, \n\tdiagnosis TEXT, \n\tradius_mean FLOAT, \n\ttexture_mean FLOAT, \n\tperimeter_mean FLOAT, \n\tarea_mean FLOAT, \n\tsmoothness_mean FLOAT, \n\tcompactness_mean FLOAT, \n\tconcavity_mean FLOAT, \n\t\"concave points_mean\" FLOAT, \n\tsymmetry_mean FLOAT, \n\tfractal_dimension_mean FLOAT, \n\tradius_se FLOAT, \n\ttexture_se FLOAT, \n\tperimeter_se FLOAT, \n\tarea_se FLOAT, \n\tsmoothness_se FLOAT, \n\tcompactness_se FLOAT, \n\tconcavity_se FLOAT, \n\t\"concave points_se\" FLOAT, \n\tsymmetry_se FLOAT, \n\tfractal_dimension_se FLOAT, \n\tradius_worst FLOAT, \n\ttexture_worst FLOAT, \n\tperimeter_worst FLOAT, \n\tarea_worst FLOAT, \n\tsmoothness_worst FLOAT, \n\tcompactness_worst FLOAT, \n\tconcavity_worst FLOAT, \n\t\"concave points_worst\" FLOAT, \n\tsymmetry_worst FLOAT, \n\tfractal_dimension_worst FLOAT\n)\n\n/*

{'answer': 'The ratio of men to women that survived the Titanic is 161 men to 339 women. This can also be expressed as approximately 1:2.1 when simplified.'}

In [21]:
def get_sql_agent_response(message: str) -> str:
    state_data: State = State({"question": message})
    query: str = write_query(state_data)["query"]
    state_data["query"] = query

    query_result: dict[str, Any] = execute_query(state_data)["result"]
    state_data["result"] = query_result

    answer: str = generate_answer(state=state_data)["answer"]
    return answer

In [22]:
get_sql_agent_response(message="How many men did NOT survive the Titanic?")

[chain/start] [prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "dialect": "sqlite",
  "top_k": 10,
  "table_info": "\nCREATE TABLE breast_cancer (\n\tid BIGINT, \n\tdiagnosis TEXT, \n\tradius_mean FLOAT, \n\ttexture_mean FLOAT, \n\tperimeter_mean FLOAT, \n\tarea_mean FLOAT, \n\tsmoothness_mean FLOAT, \n\tcompactness_mean FLOAT, \n\tconcavity_mean FLOAT, \n\t\"concave points_mean\" FLOAT, \n\tsymmetry_mean FLOAT, \n\tfractal_dimension_mean FLOAT, \n\tradius_se FLOAT, \n\ttexture_se FLOAT, \n\tperimeter_se FLOAT, \n\tarea_se FLOAT, \n\tsmoothness_se FLOAT, \n\tcompactness_se FLOAT, \n\tconcavity_se FLOAT, \n\t\"concave points_se\" FLOAT, \n\tsymmetry_se FLOAT, \n\tfractal_dimension_se FLOAT, \n\tradius_worst FLOAT, \n\ttexture_worst FLOAT, \n\tperimeter_worst FLOAT, \n\tarea_worst FLOAT, \n\tsmoothness_worst FLOAT, \n\tcompactness_worst FLOAT, \n\tconcavity_worst FLOAT, \n\t\"concave points_worst\" FLOAT, \n\tsymmetry_worst FLOAT, \n\tfractal_dimension_worst FLOAT\n)\n\n/*

'The number of men who did NOT survive the Titanic is 682.'

### Using LangGraph

In [26]:
from langgraph.graph import START, StateGraph

In [27]:
graph_builder = StateGraph(State).add_sequence(
    [write_query, execute_query, generate_answer]
)
graph_builder.add_edge(START, "write_query")
graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image


display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
langchain.debug = False

for step in graph.stream({"question": "How many men did NOT survive the Titanic?"}):
    print(step)

In [ ]:
"How many employees are there?"